<a href="https://colab.research.google.com/github/SanteriMan/freecodecamp_ML_projects/blob/master/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
dataset_path = keras.utils.get_file("insurance.csv", "https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv")
dataset = pd.read_csv(dataset_path)
dataset.head()

In [ ]:
dataset['sex'] = pd.Categorical(dataset['sex'])
dataset['sex'] = dataset['sex'].cat.codes
dataset['smoker'] = pd.Categorical(dataset['smoker'])
dataset['smoker'] = dataset['smoker'].cat.codes
dataset['region'] = pd.Categorical(dataset['region'])
dataset['region'] = dataset['region'].cat.codes
dataset.head()


In [107]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")

In [ ]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

In [137]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [164]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])
optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',
                optimizer='adam',
                metrics=['mae','mse'])

In [178]:
history = model.fit(normed_train_data, train_labels, epochs= 100,validation_split = 0.2)

Epoch 1/100
27/27 [==============================] - 0s 4ms/step - loss: 23068984.0000 - mae: 2950.4658 - mse: 23068984.0000 - val_loss: 26360640.0000 - val_mae: 3237.8455 - val_mse: 26360640.0000
Epoch 2/100
27/27 [==============================] - 0s 3ms/step - loss: 23005124.0000 - mae: 2942.7117 - mse: 23005124.0000 - val_loss: 26251212.0000 - val_mae: 3230.9668 - val_mse: 26251212.0000
Epoch 3/100
27/27 [==============================] - 0s 4ms/step - loss: 22977716.0000 - mae: 2954.5371 - mse: 22977716.0000 - val_loss: 26173966.0000 - val_mae: 3232.5327 - val_mse: 26173966.0000
Epoch 4/100
27/27 [==============================] - 0s 3ms/step - loss: 22907384.0000 - mae: 2937.4185 - mse: 22907384.0000 - val_loss: 26069792.0000 - val_mae: 3209.4719 - val_mse: 26069792.0000
Epoch 5/100
27/27 [==============================] - 0s 3ms/step - loss: 22860864.0000 - mae: 2924.1321 - mse: 22860864.0000 - val_loss: 26007816.0000 - val_mae: 3212.5967 - val_mse: 26007816.0000
Epoch 6/100
27/

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
